In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model

import numpy as np
import os

app = Flask(__name__)
CORS(app)  # Enable CORS to allow communication with the React app

# Create a directory to save uploaded images
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Prepare image for model function
def prepare_image_for_model(image, img_height=128, img_width=128):
    # Convert the image to RGB if it's not already
    if image.mode != 'RGB':
        image = image.convert('RGB')
    img = image.resize((img_width, img_height), Image.LANCZOS)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file found'}), 400
    
    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(filepath)

    # Open and preprocess the image
    image = Image.open(filepath)
    processed_image = prepare_image_for_model(image)  # Example target size
    processed_image = np.array(processed_image, dtype=np.float32)
    labels =['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']
    model = load_model('./model.h5')
    
    prediction =  model.predict(processed_image)
    print('The accuracy',max(prediction))
    index = np.argmax(prediction)
    print(index)
    result = labels[index]
    print(result)
    
    
    # For now, we'll just return the shape of the preprocessed image
    return jsonify({'message': 'Image successfully uploaded', 'shape': result}), 200

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


1/1 [==============================] - 0s 114ms/step


127.0.0.1 - - [04/Jul/2024 20:45:58] "POST //upload HTTP/1.1" 200 -


The accuracy [2.3222648e-07 3.4712550e-06 1.8849684e-02 9.8114657e-01]
3
Very_Mild_Demented
1/1 [==============================] - 0s 73ms/step


...

The accuracy [3.1309085e-06 6.2638117e-07 9.9980956e-01 1.8679078e-04]
2
Non_Demented
1/1 [==============================] - 0s 64ms/step


127.0.0.1 - - [04/Jul/2024 20:46:45] "POST //upload HTTP/1.1" 200 -


The accuracy [3.3575887e-09 9.9991822e-01 7.9370708e-05 2.3818309e-06]
1
Moderate_Demented
1/1 [==============================] - 0s 68ms/step


127.0.0.1 - - [04/Jul/2024 20:48:44] "POST //upload HTTP/1.1" 200 -


The accuracy [3.1309085e-06 6.2638117e-07 9.9980956e-01 1.8679078e-04]
2
Non_Demented
